In [ ]:
import shutil
import os
import pyodbc
import pandas as pd
from datetime import datetime

def copy_and_clean_access_query():
    # Define source and destination file paths
    source_file = r'D:\DataBase Access\SHM_TMS_001_Master_Copy.accdb'

    # Connect to the Access database and export the query to an Excel file
    conn = None

    try:
        # Connect to the Access database
        conn_str = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={source_file};'
        conn = pyodbc.connect(conn_str)

        # Specify the query name
        query_name = 'qry_output_for_excel_new'

        # Identify yesterday's date for special case in DC4
        yesterday_date = (datetime.now() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')

        # Construct the SQL query without the date condition
        query = f'''
        SELECT [PO Date],
            [PO Num],
            [PO Ref],
            [Supplier Name],
            Item,
            CJ_Item,
            CJ_Description,
            Devision,
            Rec_Date,
            [Order Qty],
            Unit,
            Delivery_Status, 
            Customer, 
            DC_Name
        FROM {query_name}
        WHERE (Delivery_Status IS NULL OR Delivery_Status = 'Pending' OR Delivery_Status = 'Hold')
        AND Customer = 'CJ'
        AND CJ_Item IS NOT null
        AND Rec_Date >= #{yesterday_date}#
        '''

        # Read the query results into a DataFrame
        df = pd.read_sql(query, conn)

        df.rename(columns={'Item': 'SHM_Item'}, inplace=True)

        dc_name_mapping = {
            'CJ DC1 ราชบุรี': 'DC1',
            'CJ DC2 บางปะกง': 'DC2',
            'DC โพธาราม': 'DC1',
            'DC บางวัว 1': 'DC2',
            'DC ขอนแก่น': 'DC4',
            'DC บางวัว 2': 'TD09'
        }
        df['DC_Name'] = df['DC_Name'].replace(dc_name_mapping)

        # Step to join with the Product List to get PC_Cartons
        try:
            # Load the product list data from Access
            product_query = """
            SELECT SHM_Item, 
                   CJ_Item,
                   PC_Cartons
            FROM qry_Product_List
            """
            product_data = pd.read_sql(product_query, conn)

            print(product_data.dtypes)

            # Merge the product data with the main DataFrame
            df = pd.merge(df, product_data[['CJ_Item', 'SHM_Item', 'PC_Cartons']], on=['CJ_Item', 'SHM_Item'], how='left')

            # Check for empty DataFrame after merging
            if df.empty:
                print("No data returned after merging with product list.")
                return
            
            # Create new column for PO_Qty with condition on Unit
            df['PO_Qty'] = df.apply(lambda row: row['Order Qty'] if row['Unit'] == 'ชิ้น' else row['Order Qty'] * row['PC_Cartons'], axis=1)

            # Pivot for PO_Qty
            pivot_po_qty = df.pivot_table(
                index=['SHM_Item','CJ_Item'],
                columns=['DC_Name'],
                values='PO_Qty',
                aggfunc='sum',
                fill_value=0
            ).reset_index()

        
            # Pivot for Min Rec_Date
            pivot_min_del_date = df.pivot_table(
                index=['SHM_Item','CJ_Item'],
                columns=['DC_Name'],
                values='Rec_Date',
                aggfunc='min'
            ).reset_index()

            # Merge both pivot tables on 'SHM_Item' and 'CJ_Item'
            pivoted_df = pivot_po_qty.merge(pivot_min_del_date, on=['SHM_Item', 'CJ_Item'], suffixes=('_fromPivotQty', '_fromPivotMindate'))

            # Rename columns
            pivot_po_qty_columns = [f'PO_Qty_to_{col}' if col not in ['SHM_Item', 'CJ_Item'] else col for col in pivot_po_qty.columns]
            pivot_min_del_date_columns = [f'Min_del_date_to_{col}' if col not in ['SHM_Item', 'CJ_Item'] else col for col in pivot_min_del_date.columns]

            pivoted_df.columns = pivot_po_qty_columns[:2] + pivot_po_qty_columns[2:] + pivot_min_del_date_columns[2:]

            print(pivoted_df.dtypes)

        except Exception as e:
            print(f"Error accessing product list: {e}")
            return

        # Define the Excel file path to save the output as requested
        excel_file_path = r'D:\Data for Stock Report\cleaned_PO_pending_other.xlsx'

        # Check if the DataFrame is not empty
        if not df.empty:
            print("Querying Process has been completed. You can now open the Excel file.")
        else:
            print("No data was returned from the query. Please check the database and query conditions.")

    except Exception as e:
        print(f"Error accessing the database or exporting to Excel: {e}")
    finally:
        # Close the database connection
        if conn is not None:
            conn.close()
            print("Connection to Access database closed.")

        # Save the DataFrame to an Excel file
        with pd.ExcelWriter(excel_file_path, mode='w') as writer:
            pivoted_df.to_excel(writer, sheet_name='Pivot All PO pending', index=False)
            df.to_excel(writer, sheet_name='cleaned data', index=False)
        
        print(f"PO pending Foods and Non-Foods report has been saved to: {excel_file_path}")

# Call the function
if __name__ == "__main__":
    copy_and_clean_access_query()
